In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
from imgaug import augmenters as iaa
from YOLO_API import *
from keras.optimizers import Adam
import time
import cv2

Using TensorFlow backend.


In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [4]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

In [5]:
anchor_boxes

[0, 0, 9, 8, 17, 15, 30, 25, 43, 66, 45, 31, 62, 43, 87, 99, 91, 56]

In [6]:
train_size=0.8
train_valid_split = int(0.8 * len(all_anno))
np.random.shuffle(all_anno)
train_anno = all_anno[:train_valid_split]
valid_anno = all_anno[train_valid_split:]
max_boxes = max([len(anno['object']) for anno in (train_anno + valid_anno)])

In [7]:
len(train_anno)

309

In [8]:
train_generator = DataGenerator(
    annotations=train_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True)

In [9]:
valid_generator = DataGenerator(
    annotations=valid_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=2,
    width=416,
    height=416,
    shuffle=True)

In [10]:
train_model, infer_mode = YoloV3(
    numcls=len(labels),
    anchors=anchor_boxes,
    max_grid=[416, 416],
    batch_size=2,
    threshold=0.5,
    max_boxes=max_boxes)

In [ ]:
infer_mode.build((416,416))
infer_mode.summary()

In [12]:
def dummy_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(y_pred))

In [13]:
opt = Adam(lr=1e-4, clipnorm=0.001)
train_model.compile(loss=dummy_loss, optimizer=opt)

In [14]:
callback = create_callbacks()

In [15]:
%%time
history = train_model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    verbose=1,
    steps_per_epoch=len(train_generator),
    epochs=300,
    callbacks=callback
)

Epoch 1/300
306/306 [==============================] - 174s 568ms/step - loss: 52.0943 - yolo_loss_layer_1_loss: 7.3452 - yolo_loss_layer_2_loss: 13.2315 - yolo_loss_layer_3_loss: 29.1895 - val_loss: 46.0383 - val_yolo_loss_layer_1_loss: 33.5750 - val_yolo_loss_layer_2_loss: 4.6725 - val_yolo_loss_layer_3_loss: 5.4620
Epoch 2/300
306/306 [==============================] - 165s 540ms/step - loss: 7.3960 - yolo_loss_layer_1_loss: 3.6690 - yolo_loss_layer_2_loss: 0.3189 - yolo_loss_layer_3_loss: 1.0804 - val_loss: 8.0138 - val_yolo_loss_layer_1_loss: 5.1456 - val_yolo_loss_layer_2_loss: 0.1271 - val_yolo_loss_layer_3_loss: 0.4156
Epoch 3/300
306/306 [==============================] - 170s 557ms/step - loss: 6.0379 - yolo_loss_layer_1_loss: 3.3541 - yolo_loss_layer_2_loss: 0.1640 - yolo_loss_layer_3_loss: 0.1973 - val_loss: 7.1612 - val_yolo_loss_layer_1_loss: 4.6495 - val_yolo_loss_layer_2_loss: 0.0905 - val_yolo_loss_layer_3_loss: 0.1023
Epoch 4/300
306/306 [=============================

In [17]:
from keras.models import model_from_json
json_string = train_model.to_json()
with open('yolo.json', 'w') as json_file:
    json_file.write(json_string)

In [18]:
train_model.save_weights('yolo_weights.h5')

In [19]:
train_model.save('yolo.h5')

In [12]:
from keras import backend as K
K.set_learning_phase(0)

In [11]:
infer_mode.load_weights('yolo_weights.h5')
# from keras.models import load_model
# model = load_model('my_model.h5', custom_objects={'AttentionLayer': AttentionLayer})
# infer_mode.load_model('yolo.h5')

In [14]:
sess = K.get_session()

In [34]:
print(infer_mode.output)

[<tf.Tensor 'conv2d_59/BiasAdd:0' shape=(?, ?, ?, 21) dtype=float32>, <tf.Tensor 'conv2d_67/BiasAdd:0' shape=(?, ?, ?, 21) dtype=float32>, <tf.Tensor 'conv2d_75/BiasAdd:0' shape=(?, ?, ?, 21) dtype=float32>]


In [16]:
tf.train.write_graph(sess.graph_def, '', 'yolo_test.pb', as_text=False)
tf.train.write_graph(sess.graph_def, '', 'yolo_graph.pbtxt')
tf.train.Saver().save(sess, 'yolo_test.chkp')

'yolo_test.chkp'

In [17]:
from tensorflow.python.tools import freeze_graph
output_node_name = 'conv2d_59/BiasAdd,conv2d_67/BiasAdd,conv2d_75/BiasAdd'
fg = freeze_graph.freeze_graph('yolo_graph.pbtxt',
                         None, False,
                         'yolo_test.chkp',
                         output_node_name,
                         'save/restore_all',
                         'save/Const:0',
                         'yolo_test.pb',
                         True, '')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from yolo_test.chkp
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 363 variables.
INFO:tensorflow:Converted 363 variables to const ops.


In [ ]:
# from tensorflow.python.tools import optimize_for_inference_lib
# graph_def = tf.GraphDef()
# with tf.gfile.Open('yolo_test.pb', "rb") as f:
#     graph_def.ParseFromString(f.read())

# output_graph_def = optimize_for_inference_lib.optimize_for_inference(
#     graph_def, ['input_1'], [output_node_name], tf.float32.as_datatype_enum)

# with tf.gfile.GFile('yolo_test.pb', "wb") as f:
#     f.write(output_graph_def.SerializeToString())

# # Strip Const nodes.
# for i in reversed(range(len(graph_def.node))):
#     if graph_def.node[i].op == 'Const':
#         del graph_def.node[i]

In [21]:
tf.train.write_graph(fg, '', 'yolo_test1.pb', as_text=False)

'yolo_test1.pb'

In [22]:
net = cv2.dnn.readNetFromTensorflow('yolo_test1.pb')

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\dnn\src\tensorflow\tf_graph_simplifier.cpp:890: error: (-2:Unspecified error) Tensor's data type is not supported in function 'cv::dnn::dnn4_v20190621::getTensorContent'


In [30]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import tensorflow as tf
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, encoding='utf-8')
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [31]:
show_graph('frozen_inference_graph_opt.pb')

AttributeError: 'str' object has no attribute 'node'

In [14]:
# from keras import backend as K
# import tensorflow as tf

# def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
#     """
#     Freezes the state of a session into a pruned computation graph.

#     Creates a new computation graph where variable nodes are replaced by
#     constants taking their current value in the session. The new graph will be
#     pruned so subgraphs that are not necessary to compute the requested
#     outputs are removed.
#     @param session The TensorFlow session to be frozen.
#     @param keep_var_names A list of variable names that should not be frozen,
#                           or None to freeze all the variables in the graph.
#     @param output_names Names of the relevant graph outputs.
#     @param clear_devices Remove the device directives from the graph for better portability.
#     @return The frozen graph definition.
#     """
#     from tensorflow.python.framework.graph_util import convert_variables_to_constants
#     graph = session.graph
#     with graph.as_default():
#         freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
#         output_names = output_names or []
#         output_names += [v.op.name for v in tf.global_variables()]
#         # Graph -> GraphDef ProtoBuf
#         input_graph_def = graph.as_graph_def()
#         if clear_devices:
#             for node in input_graph_def.node:
#                 node.device = ""
#         frozen_graph = convert_variables_to_constants(session, input_graph_def,
#                                                       output_names, freeze_var_names)
#         return frozen_graph


# frozen_graph = freeze_session(K.get_session(),
#                               output_names=[out.op.name for out in infer_mode.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 650 variables.
INFO:tensorflow:Converted 650 variables to const ops.


In [15]:
# tf.train.write_graph(frozen_graph, '', 'yolo_test1.pb', as_text=False)

'yolo_test1.pb'

In [16]:
# cv2.dnn_registerLayer('YoloLossLayer', YoloLossLayer)

In [29]:
model = cv2.dnn.readNetFromTensorflow('frozen_inference_graph_opt.pb')

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\dnn\src\tensorflow\tf_graph_simplifier.cpp:890: error: (-2:Unspecified error) Tensor's data type is not supported in function 'cv::dnn::dnn4_v20190621::getTensorContent'


In [28]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.python.tools import optimize_for_inference_lib

graph = 'yolo_test1.pb'
with tf.gfile.FastGFile(graph,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.summary.FileWriter('logs', graph_def)

    inp_node = 'input_1'
    out_node = ['conv2d_59/BiasAdd','conv2d_67/BiasAdd','conv2d_75/BiasAdd']
#     out_node = output_node_name
    graph_def = optimize_for_inference_lib.optimize_for_inference(graph_def, [inp_node], [n for n in out_node], tf.float32.as_datatype_enum)
    graph_def = TransformGraph(graph_def, [inp_node], [n for n in out_node], ["sort_by_execution_order"])
 
    with tf.gfile.FastGFile('frozen_inference_graph_opt.pb', 'wb') as f:
        f.write(graph_def.SerializeToString())

In [13]:
import keras2onnx

In [14]:
testmode = kerareadNetFromONNXkeras(
    model=infer_mode,
    name='yolo_onnx.onnx',
    doc_string='',
    target_opset=None,
    channel_first_inputs='input_1')

In [15]:
import onnx
onnx.save_model(testmode, 'test_yolo.onnx')

In [16]:
test = cv2.dnn.readNetFromONNX('test_yolo.onnx')

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\dnn\src\layers\padding_layer.cpp:33: error: (-215:Assertion failed) params.has("paddings") in function 'cv::dnn::PaddingLayerImpl::PaddingLayerImpl'


In [12]:
import glob
import os

In [13]:
testimgpath = '..\data\Test_images'
file_types = ('\*.jpg', '\*.jpeg')
test_images = []
for files in file_types:
    test_images.extend(glob.glob(testimgpath+files))

In [14]:
test_images

['..\\data\\Test_images\\icc02-003.jpeg',
 '..\\data\\Test_images\\icc02-005.jpeg',
 '..\\data\\Test_images\\icc02-006.jpeg',
 '..\\data\\Test_images\\icc02-007.jpeg',
 '..\\data\\Test_images\\icc02-008.jpeg']

In [80]:
# img_batch = np.zeros((5, 416, 416, 3))
# for i in range(len(test_images)):
#     imgr = cv2.imread(test_images[i])
#     imgrs = cv2.resize(imgr, (416,416))
#     img_batch[i] = np.expand_dims(imgrs, axis=0)

In [83]:
batch_pred = infer_mode.predict_on_batch(img_batch)


In [93]:
batch_pred[0][1].shape

(13, 13, 21)

In [42]:
infer_mode.input

<tf.Tensor 'input_1:0' shape=(?, ?, ?, 3) dtype=float32>

In [17]:
img = cv2.imread(test_images[0])
# imgrs = cv2.resize(img, (416, 416))

In [89]:
pred = infer_mode.predict(np.expand_dims(imgrs, axis=0))[0]

In [132]:
pred[0].shape

(13, 13, 21)

In [98]:
from scipy.special import expit

In [19]:
# class BoundBox:
#     def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
#         self.xmin = xmin
#         self.ymin = ymin
#         self.xmax = xmax
#         self.ymax = ymax
        
#         self.c       = c
#         self.classes = classes

#         self.label = -1
#         self.score = -1

#     def get_label(self):
#         if self.label == -1:
#             self.label = np.argmax(self.classes)
        
#         return self.label
    
#     def get_score(self):
#         if self.score == -1:
#             self.score = self.classes[self.get_label()]
            
#         return self.score    
cv2.imshow('img',img)
cv2.waitKey(0)

27

In [15]:
def preprocess_input(image, net_h, net_w):
    new_h, new_w, _ = image.shape

    # determine the new size of the image
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)//new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)//new_h
        new_h = net_h

    # resize the image to the new size
    resized = cv2.resize(image[:,:,::-1]/255., (new_w, new_h))

    # embed the image into the standard letter box
    new_image = np.ones((net_h, net_w, 3)) * 0.5
    new_image[(net_h-new_h)//2:(net_h+new_h)//2, (net_w-new_w)//2:(net_w+new_w)//2, :] = resized
    new_image = np.expand_dims(new_image, 0)

    return new_image

In [130]:
def generate_prediction_boxes(pred, anchors, thresh, height, width):
    pred_height, pred_width = pred.shape[:2]
    num_boxes = 3
    prediction_out = pred.reshape((pred_height, pred_width, num_boxes, -1))
    num_classes = pred.shape[-1]-5
    
    boxes = []
    pred[...,:2] = expit(pred[...,:2])
    pred[...,4] = expit(pred[...,4])
    pred[..., 5:] = pred[...,4][...,np.newaxis] * softmax(pred[...,5:])
    
    for i in range(pred_height*pred_width):
        row, col = i//pred_width, i%pred_width
        for j in range(num_boxes):
            objectness_score = pred[row, col, j, 4]
            if (objectness_score <= thresh):
                continue
            x, y, w, h = pred[row, col, j, :4]
            x = (col + x) / pred_width
            y = (row + y) / pred_height
            w = anchors[2 * j + 0] * np.exp(w) / width
            h = anchors[2 * j + 1] * np.exp(h) / height
            classes = pred[row, col, j, 5:]
            box = BoundBox(
                xmin=x-w/2, 
                ymin=y-h/2, 
                xmax=y+h/2,
                ymax=y+h/2,
                c = objectness,
                classes=classes)
            boxes.append(box)
    return boxes

In [148]:
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    if (float(net_w)/image_w) < (float(net_h)/image_h):
        new_w = net_w
        new_h = (image_h*net_w)/image_w
    else:
        new_h = net_w
        new_w = (image_w*net_h)/image_h
        
    for i in range(len(boxes)):
        x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
        
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)

In [150]:
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
        
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i].classes[c] == 0: continue

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0

In [152]:
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  
    
    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    
    union = w1*h1 + w2*h2 - intersect
    
    return float(intersect) / union

In [154]:
def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3   

In [176]:
boxes = []
for j in range(len(pred)):
    yolo_anchors = anchor_boxes[(2-j)*6:(3-j)*6]
    boxes += generate_prediction_boxes(pred[j],yolo_anchors, 0.5, 416,416)

IndexError: too many indices for array

In [168]:
a, b = expit(pred[...,:2]), expit(pred[...,4])

In [169]:
h, w = pred[0].shape[:2]
t = pred[0].reshape((h, w, 3, -1))
nbcls = t.shape[-1]

In [170]:
def softmax(x, axis=-1):
    x = x - np.amax(x, axis, keepdims=True)
    e_x = np.exp(x)
    return e_x/e_x.sum(axis, keepdims=True)

In [171]:
test = b[...,np.newaxis] * softmax(pred[...,5:])

In [172]:
pred_org= pred.copy()

In [173]:
test *= pred[...,5:] > 0.5

In [174]:
pred_height, pred_width = pred.shape[:2]
num_boxes = 3
prediction_out = pred.reshape((pred_height, pred_width, num_boxes, -1))
num_classes = pred.shape[-1]-5

boxes = []
pred[...,:2] = expit(pred[...,:2])
pred[...,4] = expit(pred[...,4])
pred[..., 5:] = pred[...,4][...,np.newaxis] * softmax(pred[...,5:])

In [175]:
for i in range(pred_height*pred_width):
    row, col = i//pred_width, i%pred_width
    for j in range(num_boxes):
        objectness_score = pred[row, col, j, 4]
        if (objectness_score <= 0.5):
            continue
        x, y, w, h = pred[row, col, j, :4]
        x = (col + x) / pred_width
        y = (row + y) / pred_height
        w = anchor_boxes[2 * j + 0] * np.exp(w) / 416
        h = anchor_boxes[2 * j + 1] * np.exp(h) / 416
        classes = pred[row, col, j, 5:]
        box = ia.BoundingBox(
            x1=x-w/2, 
            x2=y-h/2, 
            y1=y+h/2,
            y2=y+h/2,
            label=classes)
        boxes.append(box)
boxes

[BoundingBox(x1=-0.0585, y1=1.3034, x2=0.0479, y2=1.3034, label=[0.0420436  0.0420436  0.04312663 0.0420436  0.0420436  0.0420436
  0.0420436  0.0420436  0.0420436  0.0420436  0.04330043 0.0420436
  0.0420436  0.0420436  0.0420436  0.0420436 ]),
 BoundingBox(x1=-165634766.1468, y1=165634767.3917, x2=0.0519, y2=165634767.3917, label=[0.041997   0.041997   0.041997   0.041997   0.041997   0.041997
  0.041997   0.041997   0.041997   0.041997   0.04508253 0.041997
  0.041997   0.041997   0.041997   0.041997  ]),
 BoundingBox(x1=-765496.2237, y1=765497.4686, x2=-2.2520, y2=765497.4686, label=[0.041997   0.041997   0.041997   0.041997   0.041997   0.041997
  0.041997   0.041997   0.041997   0.041997   0.04508253 0.041997
  0.041997   0.041997   0.041997   0.041997  ]),
 BoundingBox(x1=0.1288, y1=0.6745, x2=0.6745, y2=0.6745, label=[0.04199701 0.041997   0.041997   0.041997   0.04508251 0.041997
  0.041997   0.041997   0.041997   0.041997   0.041997   0.041997
  0.041997   0.041997   0.041997

In [162]:
all_boxes = copy.deepcopy(boxes)
xScale, yScale = float(416) / 416, float(416) / 416
box_list = []
for box in boxes:
    scale_box = ia.BoundingBox(
        x1=int(np.round(box.x1 * xScale)),
        x2=int(np.round(box.x2 * xScale)),
        y1=int(np.round(box.y1 * yScale)),
        y2=int(np.round(box.y2 * yScale)),
        label=box.label)
    box_list.append(scale_box)
bboi = ia.BoundingBoxesOnImage(box_list, (new_width, new_height))

AttributeError: 'BoundBox' object has no attribute 'x1'

In [155]:
do_nms(boxes, 0.5)

ZeroDivisionError: float division by zero